In [17]:
# Source: https://www.tensorflow.org/tutorials/quickstart/beginner

In [1]:
%load_ext tensorboard
import tensorflow as tf 
import datetime

# Clear any logs from previous runs
! rm -rf ./logs/


In [2]:
# Import the mnist dataset

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

(60000, 28, 28)

In [22]:
# Build the tf.keras.Sequential model by stacking layers. 
# Choose an optimizer and loss function for training:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [23]:
# For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.23151295, -0.43886524, -0.12244322, -0.31110114,  0.8436471 ,
         0.4034411 , -0.23595598,  0.6900139 ,  0.5120231 ,  0.28618404]],
      dtype=float32)

In [24]:
# The tf.nn.softmax function converts these logits to "probabilities" for each class:
tf.nn.softmax(predictions).numpy()

# The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.


In [26]:
loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [28]:
# Log stats
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [29]:
# The Model.fit method adjusts the model parameters to minimize the loss:

In [30]:
model.fit(x_train, y_train, epochs=5,callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2933 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1443 - accuracy: 0.9564
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1099 - accuracy: 0.9662
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0894 - accuracy: 0.9721
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0793 - accuracy: 0.9750


In [31]:
# The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [32]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0756 - accuracy: 0.9758


[0.07562652230262756, 0.9757999777793884]

In [33]:
# The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

# If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [34]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [35]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.17929177e-08, 3.55676155e-09, 2.01888706e-05, 4.18941578e-04,
        8.46635817e-10, 2.35380355e-07, 8.56099859e-13, 9.99545157e-01,
        2.22601420e-06, 1.30948702e-05],
       [6.86594717e-07, 3.26604786e-05, 9.99650955e-01, 1.96414912e-04,
        8.54199930e-14, 1.20119685e-05, 4.69812403e-05, 1.93948468e-11,
        6.02822038e-05, 1.43801290e-11],
       [6.51881891e-08, 9.99434173e-01, 2.57560256e-04, 4.20988181e-06,
        8.58426029e-06, 2.46702166e-06, 5.10810642e-05, 2.14019514e-04,
        2.72090620e-05, 5.24665950e-07],
       [9.99141216e-01, 7.08573001e-12, 3.83837578e-05, 4.66181660e-07,
        5.34491882e-08, 8.35050469e-06, 7.68781523e-04, 4.31183162e-06,
        2.04068655e-08, 3.84560117e-05],
       [5.99126111e-07, 9.25269728e-09, 1.44049118e-05, 6.03515105e-08,
        9.99471128e-01, 1.18365293e-07, 1.30473436e-05, 9.78316602e-05,
        7.76702200e-06, 3.95073090e-04]], dtype=float32)>

In [36]:
%tensorboard --logdir logs/fit